In [1]:
import gdspy
import numpy as np

In [2]:
gdsii = gdspy.GdsLibrary()
file = '/Users/wendy/Desktop/Wendy-qiskit-code/export_test_0506_LL.gds'
gdsii.read_gds(file, units = 'import')

gds_new = gdspy.GdsLibrary(unit = 1e-3, precision = 1e-9)
# gds_new.cells.clear()

In [3]:
top = gds_new.new_cell('TOP')



In [4]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60
chip_edge_layer = int(703)
text_layer1 = 98

layer_numbers  = [qubit_layer,junction_layer,ab_layer,ab_square_layer,junction_area_layer]

In [5]:
qubit_layer_name = 'TOP_M1'
junction_name = 'TOP_SE1'
ab_name = 'TOP_BR'
ab_square_name = 'TOP_RR'
junction_area_name = 'TOP_SE1_JJ'
chip_edge_name = 'TOP_ChipEdge'



In [6]:
qubit_cell = gdspy.Cell(qubit_layer_name)
junction_cell = gdspy.Cell(junction_name)
ab_cell = gdspy.Cell(ab_name)
ab_square_cell = gdspy.Cell(ab_square_name)
junction_area_cell = gdspy.Cell(junction_area_name)
chip_edge_cell = gdspy.Cell(chip_edge_name)


In [7]:
cell_names = [qubit_cell, junction_cell, ab_cell,ab_square_cell]#, junction_area_cell]

In [8]:
data_type = 0

In [9]:
def snap_to_grid(coord, grid_size):
    return np.round(coord / grid_size) * grid_size

In [10]:
qubit_layer_grid = 50*1e-6
junction_layer_grid = 5*1e-6

In [11]:
polys = gdsii.top_level()[0].get_polygons(by_spec = True)

In [12]:
keys = list(polys.keys())

In [13]:
keys

[(5, 10), (5, 11), (5, 0), (20, 10), (60, 10), (30, 10), (31, 10)]

In [14]:
datatypes = {}
for i in keys:
    try:
        print(datatypes[i[0]])
    except:
        datatypes[i[0]] = [i[1]]
    else:
        datatypes[i[0]].append(i[1])


[10]
[10, 11]


In [15]:
datatypes

{5: [10, 11, 0], 20: [10], 60: [10], 30: [10], 31: [10]}

In [16]:
round

<function round(number, ndigits=None)>

In [17]:
chip_edge_layer

703

In [18]:
for i, layer in enumerate(layer_numbers):
    for j in datatypes[layer]:
        source_polyons = polys.get((layer,j),[])

        for polygon in source_polyons:
            if layer ==5:
                polygon = snap_to_grid(polygon, qubit_layer_grid)
            elif layer == 20:
                polygon = snap_to_grid(polygon, junction_layer_grid)
            p =gdspy.Polygon(polygon, layer = int(layer), datatype = 0)
            # print(cell_names[i])
            top.add(p)

In [19]:
junction_area_cell.add(gdspy.Polygon(polygon[:-1], layer = int(junction_area_layer)))

In [20]:
p = gdspy.Polygon(polygon[:-1], layer = 0)
chip_edge_cell.add(p)

In [21]:
p

In [22]:
rectangle = gdspy.Polygon(np.array([[-5,5],[5,5],[5,-5],[-5,-5]]), layer = int(chip_edge_layer))
top.add(rectangle)

In [23]:
gds_new.write_gds('compatible_try2.gds')

In [24]:
# ref1 = gdspy.CellReference(qubit_cell)
# ref2 = gdspy.CellReference(junction_cell)
# ref3 = gdspy.CellReference(ab_cell)
# ref4 = gdspy.CellReference(ab_square_cell)
# ref5 = gdspy.CellReference(junction_area_cell)
# ref6 = gdspy.CellReference(chip_edge_cell)

# top.add(ref1)
# top.add(ref2)
# top.add(ref3)
# top.add(ref4)
# top.add(ref5)
# top.add(ref6)

In [25]:
qubit_cell.get_polygons(by_spec = True)

{}

In [33]:
htest1 = gdspy.Text('Email',1,(4,0))
htest2 = gdspy.Text('wendywan@stanford.edu',1, (3,0))
htest3 = gdspy.Text('Jc = 0.1uA/cm2',1,(2,0))

In [26]:
# top = qubit_cell = gds_new.new_cell('TOP')
# rec = gdspy.Polygon(np.array([[-500.,500.],[500.,500.],[500.,-500.],[-500.,-500.]]), layer = int(0))
# top.add(rec)

In [27]:
gds_new.write_gds('compatible_try21.gds')